## Loan Data - Train and Test Set Preparation and Selection ##

In [10]:
import numpy as np
import pandas as pd

Read in the data.

In [11]:
loans_2007_2015 = pd.read_csv('Data/lc_loan.csv', low_memory=False)
loans_2016_2017 = pd.read_csv('Data/lc_2016_2017.csv', low_memory=False)

View the available loan status categories in both data sets.

In [12]:
loans_2007_2015['loan_status'].value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [13]:
loans_2016_2017['loan_status'].value_counts()

Current               565523
Fully Paid            130718
Charged Off            37197
Late (31-120 days)     15354
In Grace Period         6634
Late (16-30 days)       3876
Default                   36
Name: loan_status, dtype: int64

**We are interested in distinguishing loans which definitely turned sour from the confirmed good ones. That is why we restrict our attention to the following two categories of the considered loans: the fully paid ones and those that have been charged off, since other categories are temporal in nature (grace period, late, default), non-standard (credit policy not met) or undetermined (current).**

Select either good (fully paid) or really bad (charged off) loans.

In [14]:
selected_loan_categories = ['Fully Paid', 'Charged Off']
selected_loans_2007_2015 = loans_2007_2015.loc[loans_2007_2015['loan_status'].\
                                               isin(selected_loan_categories), :].copy()
selected_loans_2016_2017 = loans_2016_2017.loc[loans_2016_2017['loan_status'].\
                                               isin(selected_loan_categories), :].copy()

View the shape of these new data sets.

In [15]:
selected_loans_2007_2015.shape, selected_loans_2016_2017.shape

((252971, 74), (167915, 72))

The following two columns are not in the second data frame and need to be dropped.

In [16]:
extra_columns = set(selected_loans_2007_2015.columns.values.tolist()) - \
                set(selected_loans_2016_2017.columns.values.tolist())
extra_columns = list(extra_columns)
extra_columns

['url', 'open_il_6m']

In [17]:
selected_loans_2007_2015 = selected_loans_2007_2015.drop(columns=extra_columns)

Confirm that now both data sets have the same columns in the same order.

In [18]:
(selected_loans_2007_2015.columns == selected_loans_2016_2017.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

Convert the **loan_status** column into a binary zero-one indicator, where 0 corresponds to good and 1 to bad loans.

In [19]:
selected_loans_2007_2015.loc[selected_loans_2007_2015['loan_status'].\
                             isin(['Fully Paid']), 'loan_status'] = 0
selected_loans_2007_2015.loc[selected_loans_2007_2015['loan_status'].\
                             isin(['Charged Off']), 'loan_status'] = 1
selected_loans_2007_2015['loan_status'] = selected_loans_2007_2015['loan_status'].\
                                          astype(np.uint8)

selected_loans_2016_2017.loc[selected_loans_2016_2017['loan_status'].\
                             isin(['Fully Paid']), 'loan_status'] = 0
selected_loans_2016_2017.loc[selected_loans_2016_2017['loan_status'].\
                             isin(['Charged Off']), 'loan_status'] = 1
selected_loans_2016_2017['loan_status'] = selected_loans_2016_2017['loan_status'].\
                                          astype(np.uint8)

View the loan status distribution.

In [20]:
def ColumnValueDistribution(column):
    val_count = column.value_counts()
    val_percentage = column.value_counts(normalize=True)
    column_info = pd.DataFrame([val_count, val_percentage]).T
    column_info.columns = ['Count', 'Percentage']
    column_info['Count'] = column_info['Count'].astype(np.int32)
    column_info['Percentage'] = column_info['Percentage'].values.round(4)*100
    column_info = column_info.rename_axis('loan status', axis='index')
    
    return column_info

In [21]:
ColumnValueDistribution(selected_loans_2007_2015['loan_status'])

,Count,Percentage
loan status,,
0,207723,82.11
1,45248,17.89


In [22]:
ColumnValueDistribution(selected_loans_2016_2017['loan_status'])

,Count,Percentage
loan status,,
0,130718,77.85
1,37197,22.15


**Clearly, the loan status categories are not well ballanced which will have to be taken into account at a later stage. However, the ratio of bad to good loans is roughly equivalent in both data sets, though unfortunately not the same, <br> and is approximately 1 to 4.**

Define the train and test data sets and save them for further analysis.

In [23]:
selected_loans_2007_2015.to_csv('Data/train_data_raw.csv', index=False)
selected_loans_2016_2017.to_csv('Data/test_data_raw.csv', index=False)